In [4]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
import torch
import pickle as pkl


<h1>Trying a pretrained model (Resnet152V2)</h1>

In [5]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"

In [6]:
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        len_classes = len(image_folder.class_to_idx)
        self.images = [image[0] for image in image_folder]
        self.labels = [torch.Tensor(self.labeler(image[1], len_classes)) for image in image_folder]
        self.class_dict = image_folder.class_to_idx

    def labeler(self, label, len_classes):
        result = [0] * len_classes
        result[label] = 1
        return result

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
    


In [7]:
dataset = DatasetAppels(dataset_path, T.ToTensor())

dataset

In [8]:
# Resnet variants:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=False)
model.eval()

Using cache found in C:\Users\nilsm/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\nilsm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\nilsm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

for image in dataset.images:
    image = preprocess(image)

C:\Users\nilsm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>